In [1]:
import pandas as pd
import numpy as np

import re
import requests
from utils.data_utils import missing_value_summary

In [2]:
data = pd.read_csv("data/metacritic_tv_shows.csv")

In [7]:
data.head(5)

,id,title,releaseDate,seasonCount,rating,genres,description,duration,tagline,metascore,metascore_count,metascore_sentiment,userscore,userscore_count,userscore_sentiment,created_by,production_companies,director,writer,top_cast
0,1000358361,Planet Earth: Blue Planet II,2017-10-29,1.0,TV-G,Documentary,"Airing simultaneously on AMC, BBC America, IFC...",50.0,Take a deep breath,97.0,7,Universal acclaim,82,178,Universal acclaim,NaN,"BBC Natural History Unit (NHU),BBC Studios,BBC...",James Honeyborne,NaN,"David Attenborough,Peter Drost,Roger Munns,Rog..."
1,1000359012,America to Me,2018-08-26,1.0,TV-14,Documentary,The 10-part documentary series from Steve Jame...,60.0,NaN,96.0,9,Universal acclaim,59,75,Mixed or average,NaN,"Participant,Kartemquin Films,Starz,Starz,Nolo ...","Bing Liu,Kevin Shaw,Steve James,Rebecca Parrish",NaN,"Kendale McCoy,Charles Donalson Jr.,Ke'Shawn Ku..."
2,1000357720,Planet Earth II,2016-11-06,1.0,TV-G,Documentary,"Narrated by David Attenborough, the sequel to ...",50.0,A new world revealed,96.0,10,Universal acclaim,92,242,Universal acclaim,NaN,"BBC Natural History Unit (NHU),BBC America,Zwe...","Justin Anderson,Ed Charles,Elizabeth White,Emm...",Elizabeth White,"David Attenborough,Chadden Hunter,Gordon Bucha..."
3,1000302375,The Staircase,2004-06,1.0,TV-MA,"Documentary,Crime,Drama",An 8-part documentary series about the celebra...,NaN,Did He Do It?,95.0,9,Universal acclaim,71,62,Generally favorable,NaN,"Maha Productions,ABC News,Docurama,Netflix,Net...",Jean-Xavier de Lestrade,"Jean-Xavier de Lestrade,Nathalie Sobania","Michael Peterson,David Rudolf,Ron Guerette,Mar..."
4,1000366530,The U.S. and the Holocaust,2022-09-18,1.0,TV-14,"Documentary,History","Narrated by Peter Coyote, the three-part docum...",133.0,NaN,96.0,10,Universal acclaim,53,65,Mixed or average,NaN,"Florentine Films,Public Broadcasting Service (...","Sarah Botstein,Ken Burns,Lynn Novick",Geoffrey C. Ward,"Peter Coyote,Daniel Mendelsohn,Peter Hayes,Deb..."


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3330 entries, 0 to 3329
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3330 non-null   int64  
 1   title                 3330 non-null   object 
 2   releaseDate           3317 non-null   object 
 3   seasonCount           3135 non-null   float64
 4   rating                2807 non-null   object 
 5   genres                3328 non-null   object 
 6   description           3329 non-null   object 
 7   duration              3002 non-null   float64
 8   tagline               1722 non-null   object 
 9   metascore             3314 non-null   float64
 10  metascore_count       3330 non-null   int64  
 11  metascore_sentiment   3314 non-null   object 
 12  userscore             3330 non-null   int64  
 13  userscore_count       3330 non-null   int64  
 14  userscore_sentiment   2871 non-null   object 
 15  created_by           

In [10]:
missing_value_summary(data)

,Number of Missing Values,Percent of Total Values
created_by,1618,48.59
tagline,1608,48.29
director,1139,34.20
rating,523,15.71
userscore_sentiment,459,13.78
duration,328,9.85
writer,243,7.30
seasonCount,195,5.86
metascore_sentiment,16,0.48
metascore,16,0.48


In [3]:
data[data["metascore"].isna()]

,id,title,releaseDate,seasonCount,rating,genres,description,duration,tagline,metascore,metascore_count,metascore_sentiment,userscore,userscore_count,userscore_sentiment,created_by,production_companies,director,writer,top_cast
3310,1000300744,The Underground,2006-09-14,1.0,TV-MA,Comedy,"Damon Wayans is at it again! Damon Wayans, a c...",30.0,WARNING: Seriously funny skit.,NaN,12,NaN,0,0,NaN,NaN,"Amara Production Films,Showtime Networks","Damon Wayans,Damien Dante Wayans","Damon Wayans,Will Bowles,Mikey Day,Josh Flaum,...","Damon Wayans,Edi Patterson,Mikey Day,Damon Way..."
3311,1000331351,The 39 Steps,2008-12-28,NaN,TV-PG,"Adventure,Crime,Drama,Mystery,Thriller","Rupert Penry-Jones (Burn Up, Spooks) stars as ...",90.0,NaN,NaN,8,NaN,0,0,NaN,NaN,"British Broadcasting Corporation (BBC),British...",James Hawes,"John Buchan,Lizzie Mickery","Rupert Penry-Jones,Lydia Leonard,David Haig,Pa..."
3315,1000301823,Wanted,2005-07-31,1.0,NaN,"Crime,Drama,Thriller","The harsh, unforgiving streets of Los Angeles ...",NaN,NaN,NaN,8,NaN,0,1,NaN,NaN,"Badlands Entertainment,Spelling Television,Tur...","Guy Norman Bee,Félix Enríquez Alcalá,Thomas J....","Louis St. Clair,Jorge Zamacona,Robert Palm,Kim...","Gary Cole,Ryan Hurst,Rashida Jones,Benjamín Be..."
3316,1000300985,The Surreal Life,2003-01-09,9.0,TV-14,"Comedy,Reality-TV","Once, they were superstars. Now, they are shov...",22.0,See what happens with celebrities stop faking ...,NaN,7,NaN,0,0,NaN,NaN,"51 Minds Entertainment,Brass Ring Productions,...",Robert Sizemore,Mark Cronin,"Charo,Dave Coulier,Flavor Flav,Jordan Knight,B..."
3317,1000302417,"Tsunami, The Aftermath",2006,2.0,TV-MA,Drama,BBC and HBO Films bring the touching story of ...,90.0,NaN,NaN,20,NaN,0,1,NaN,NaN,"British Broadcasting Corporation (BBC),HBO Fil...",Bharat Nalluri,Abi Morgan,"Hugh Bonneville,Gina McKee,Samrit Machielsen,G..."
3318,1000303082,The Baby Borrowers,2008-06-25,1.0,NaN,Reality-TV,"Most teenagers can't wait for adulthood, and f...",60.0,NaN,NaN,8,NaN,98,4,Universal acclaim,NaN,"Love Productions,National Broadcasting Company...",NaN,"Ann Beattie,Richard McKerrow","Craig Zimmerman,Xavier Amador,Dave Schaich,Che..."
3320,1000358205,Tin Star,2017-09-07,3.0,TV-MA,"Crime,Drama,Thriller",The arrival of a new oil refinery led by Eliza...,60.0,Vengeance comes with a price.,NaN,7,NaN,56,48,Mixed or average,NaN,"Kudos Film and Television,Amazon Prime Video,B...","Justin Chadwick,Alice Troughton,Gilles Bannier","Rowan Joffe,Chris Hurford,Tom Butterworth,Nath...","Tim Roth,Genevieve O'Reilly,Abigail Lawrie,Oli..."
3321,1000302528,"Tim and Eric Awesome Show, Great Job!",2007-02-11,6.0,TV-14,"Comedy,Music","From Tim and Eric, the creators of Adult Swim'...",11.0,NaN,NaN,24,NaN,81,70,Universal acclaim,NaN,"Tim and Eric,Abso Lutely Productions,Williams ...","Tim Heidecker,Eric Wareheim,Jonathan Krisel","Tim Heidecker,Eric Wareheim,Jon Mugar,Doug Lus...","Tim Heidecker,Eric Wareheim,Bob Odenkirk,John ..."
3322,1000338101,Top Gear,2002-10-20,34.0,TV-PG,"Adventure,Comedy,Sport,Talk-Show","More than just a show to review new cars, Top ...",60.0,Bring on the Bangers and Mash!,NaN,9,NaN,79,430,Generally favorable,NaN,"British Broadcasting Corporation (BBC),BBC Ame...","Brian Klein,Phil Churchward,Nigel Simpkiss,Jon...",NaN,"Jeremy Clarkson,Richard Hammond,James May,The ..."
3323,1000341758,Reckless,2014-06-29,2.0,NaN,"Crime,Drama","In Charleston, South Carolina, former Chicago ...",60.0,NaN,NaN,15,NaN,0,0,NaN,Dana Stevens,"Sander/Moses Productions,CBS Television Studio...",John Gray,"Dana Stevens,Jon Worley,Devon Greggory,Corey M...","Kim Wayans,Michael Gladis,Gregory Harrison,Ann..."


In [4]:
# Remove rows where BOTH metascore is NaN AND userscore_count is 0
data = data[~(data['metascore'].isna() & (data['userscore_count'] == 0))]

In [14]:
missing_value_summary(data)

,Number of Missing Values,Percent of Total Values
created_by,1613,48.53
tagline,1604,48.26
director,1139,34.27
rating,520,15.64
userscore_sentiment,453,13.63
duration,328,9.87
writer,243,7.31
seasonCount,194,5.84
releaseDate,13,0.39
top_cast,12,0.36


In [15]:
data[data["metascore"].isna()]

,id,title,releaseDate,seasonCount,rating,genres,description,duration,tagline,metascore,metascore_count,metascore_sentiment,userscore,userscore_count,userscore_sentiment,created_by,production_companies,director,writer,top_cast
3315,1000301823,Wanted,2005-07-31,1.0,NaN,"Crime,Drama,Thriller","The harsh, unforgiving streets of Los Angeles ...",NaN,NaN,NaN,8,NaN,0,1,NaN,NaN,"Badlands Entertainment,Spelling Television,Tur...","Guy Norman Bee,Félix Enríquez Alcalá,Thomas J....","Louis St. Clair,Jorge Zamacona,Robert Palm,Kim...","Gary Cole,Ryan Hurst,Rashida Jones,Benjamín Be..."
3317,1000302417,"Tsunami, The Aftermath",2006,2.0,TV-MA,Drama,BBC and HBO Films bring the touching story of ...,90.0,NaN,NaN,20,NaN,0,1,NaN,NaN,"British Broadcasting Corporation (BBC),HBO Fil...",Bharat Nalluri,Abi Morgan,"Hugh Bonneville,Gina McKee,Samrit Machielsen,G..."
3318,1000303082,The Baby Borrowers,2008-06-25,1.0,NaN,Reality-TV,"Most teenagers can't wait for adulthood, and f...",60.0,NaN,NaN,8,NaN,98,4,Universal acclaim,NaN,"Love Productions,National Broadcasting Company...",NaN,"Ann Beattie,Richard McKerrow","Craig Zimmerman,Xavier Amador,Dave Schaich,Che..."
3320,1000358205,Tin Star,2017-09-07,3.0,TV-MA,"Crime,Drama,Thriller",The arrival of a new oil refinery led by Eliza...,60.0,Vengeance comes with a price.,NaN,7,NaN,56,48,Mixed or average,NaN,"Kudos Film and Television,Amazon Prime Video,B...","Justin Chadwick,Alice Troughton,Gilles Bannier","Rowan Joffe,Chris Hurford,Tom Butterworth,Nath...","Tim Roth,Genevieve O'Reilly,Abigail Lawrie,Oli..."
3321,1000302528,"Tim and Eric Awesome Show, Great Job!",2007-02-11,6.0,TV-14,"Comedy,Music","From Tim and Eric, the creators of Adult Swim'...",11.0,NaN,NaN,24,NaN,81,70,Universal acclaim,NaN,"Tim and Eric,Abso Lutely Productions,Williams ...","Tim Heidecker,Eric Wareheim,Jonathan Krisel","Tim Heidecker,Eric Wareheim,Jon Mugar,Doug Lus...","Tim Heidecker,Eric Wareheim,Bob Odenkirk,John ..."
3322,1000338101,Top Gear,2002-10-20,34.0,TV-PG,"Adventure,Comedy,Sport,Talk-Show","More than just a show to review new cars, Top ...",60.0,Bring on the Bangers and Mash!,NaN,9,NaN,79,430,Generally favorable,NaN,"British Broadcasting Corporation (BBC),BBC Ame...","Brian Klein,Phil Churchward,Nigel Simpkiss,Jon...",NaN,"Jeremy Clarkson,Richard Hammond,James May,The ..."
3324,1000359281,Elite,2018-10-05,8.0,TV-MA,"Crime,Drama,Thriller","The Spanish-language drama set at Las Encinas,...",60.0,"""Til death do us part.""",NaN,8,NaN,73,186,Generally favorable,"Darío Madrona,Carlos Montero","Zeta Ficción,Netflix,Producciones Zeta,Zeta St...","Dani de la Orden,Ginesta Guindal,Lino Escalera","Darío Madrona,Carlos Montero,Jaime Vaca","Omar Ayuso,Itzan Escamilla,Miguel Bernardeau,V..."
3326,1000300177,Saturday Night Live,1975-10-11,51.0,TV-PG,"Comedy,Music","""Live from New York, it's Saturday Night!""\r\n...",90.0,"And now, from New York, the most dangerous cit...",NaN,8,NaN,65,177,Generally favorable,NaN,"NBC Studios,NBC Universal Television,Broadway ...",Dave Wilson,"Lorne Michaels,Steve Higgins,James Downey,Erik...","The Saturday Night Live Band,Don Pardo,Darrell..."
3327,1000300134,Hill Street Blues,1981-01-15,7.0,TV-14,"Crime,Drama,Mystery",This groundbreaking and award-winning NBC crim...,60.0,NaN,NaN,9,NaN,92,15,Universal acclaim,"Steven Bochco,Michael Kozoll","MTM Enterprises,MTM Productions,National Broad...",NaN,"Steven Bochco,Michael Kozoll,Jeffrey Lewis,Dav...","Daniel J. Travanti,Michael Warren,Bruce Weitz,..."
3328,1000302327,The Naked Trucker and T-Bones Show,2007-01-17,1.0,NaN,Comedy,"This subversive, comedy / variety show is base...",30.0,NaN,NaN,9,NaN,0,2,NaN,NaN,"Comedy Central,Parallel Entertainment Pictures...","Jay Karas,Linda Mendoza","Norm Hiscock,Dave Allen,David Koechner,Peter G...","Dave Allen,David Koechner,J.P. Fitting,Steve C..."


In [5]:
data = data[~data['metascore'].isna()]# removes those 10 with missing metascore

In [18]:
missing_value_summary(data)

,Number of Missing Values,Percent of Total Values
created_by,1605,48.43
tagline,1598,48.22
director,1137,34.31
rating,517,15.60
userscore_sentiment,450,13.58
duration,327,9.87
writer,242,7.30
seasonCount,194,5.85
releaseDate,13,0.39
top_cast,12,0.36


In [6]:
data['top_cast'] = data['top_cast'].fillna('No Cast')

In [7]:
data['seasonCount_was_null'] = data['seasonCount'].isna()  
data['seasonCount'] = data['seasonCount'].fillna(1)

In [74]:
#data[data["releaseDate"].isna()]

In [8]:
missing_value_summary(data)

,Number of Missing Values,Percent of Total Values
created_by,1605,48.43
tagline,1598,48.22
director,1137,34.31
rating,517,15.60
userscore_sentiment,450,13.58
duration,327,9.87
writer,242,7.30
releaseDate,13,0.39
production_companies,3,0.09
genres,2,0.06


In [9]:

api_key = 'f4cf3ff6'


In [10]:
def get_release_date(title):    
    url = f'http://www.omdbapi.com/?t={title}&apikey={api_key}'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data.get('Response') == 'True':
            return data.get('Released') or data.get('Year')
    
    return None


In [143]:
mask = data['releaseDate'].isna()
data.loc[mask, 'releaseDate'] = data.loc[mask, 'title'].apply(get_release_date)

In [11]:
data[data["releaseDate"].isna()]

,id,title,releaseDate,seasonCount,rating,genres,description,duration,tagline,metascore,...,metascore_sentiment,userscore,userscore_count,userscore_sentiment,created_by,production_companies,director,writer,top_cast,seasonCount_was_null
209,1000342313,Whoopi Goldberg Presents Moms Mabley,NaN,1.0,NaN,"Documentary,Comedy","With newly discovered archival footage, photog...",72.0,NaN,83.0,...,Universal acclaim,0,1,NaN,NaN,Whoop/One Ho Productions/Lil' Whoop Production...,Whoopi Goldberg,NaN,"Debbie Allen,Kaye Ballard,Harry Belafonte,Dick...",True
354,1000341790,The Ghost Army,NaN,1.0,TV-PG,"Documentary,History",Rick Beyer's documentary about the secret miss...,68.0,Illusion was their ultimate weapon,80.0,...,Generally favorable,0,0,NaN,NaN,"Plate of Peas Productions,PBS Distribution,PBS...",Rick Beyer,Rick Beyer,"Al Albrecht,Spike Berry,Ed Biow,Winston Church...",True
474,1000338902,Becoming Chaz,NaN,1.0,NaN,"Documentary,Biography","This honest portrayal of Chaz Bono, the only c...",80.0,NaN,78.0,...,Generally favorable,0,0,NaN,NaN,"World of Wonder Productions,Virgil Films & Ent...","Fenton Bailey,Randy Barbato",NaN,"Cher,Chaz Bono,Georganne LaPiere,Georgia Holt,...",True
556,1000361698,Philly D.A.,NaN,1.0,TV-PG,Documentary,Directors Yoni Brook and Ted Passon follow civ...,110.0,NaN,77.0,...,Generally favorable,0,2,NaN,NaN,"Department of Motion Pictures,Doc Society,All ...",NaN,Nicole Salazar,No Cast,True
607,1000341708,Manhunt (2013),NaN,1.0,TV-14,"Documentary,History,War",The documentary directed by Greg Barker and ba...,102.0,NaN,76.0,...,Generally favorable,0,0,NaN,NaN,"HBO Documentary Films,Motto Pictures,Passion P...",Greg Barker,Peter Bergen,"Osama bin Laden,Nada Bakos,Marty Martin,Cindy ...",True
641,1000347010,Nixon by Nixon: In His Own Words,NaN,1.0,NaN,Documentary,The documentary from Peter Kunhardt includes a...,71.0,40 years later his words continue to shock.,76.0,...,Generally favorable,0,0,NaN,NaN,"Kunhardt Films,HBO Max,HBO Documentary Films,H...",Peter W. Kunhardt,NaN,"Ramona Banuelos,David Brinkley,Pat Buchanan,St...",True
660,1000341832,Richard Pryor: Omit the Logic,NaN,1.0,NaN,"Documentary,Biography","In director Marina Zenovich's documentary, Lil...",83.0,NaN,75.0,...,Generally favorable,0,2,NaN,NaN,"Fresh One Productions,Tarnished Angel,Showtime...",Marina Zenovich,"P.G. Morgan,Chris A. Peterson,Marina Zenovich","David Banks,Rona Barrett,David Brinkley,Harry ...",True
1390,1000350546,Nightingale,NaN,1.0,NaN,"Crime,Drama",War veteran Peter (David Oyelowo) struggles to...,83.0,There's no escaping your own mind.,67.0,...,Generally favorable,0,0,NaN,NaN,"HBO Films,BN Films,Plan B Entertainment,Sea Sm...",Elliott Lester,Frederick Mensch,"David Oyelowo,Barlow Jacobs,Heather Storm",True
1623,1000342253,Muhammad Ali's Greatest Fight,NaN,1.0,TV-14,"Biography,Drama",Stephen Frears’ HBO film based on the book by ...,97.0,Some fights are bigger than the ring.,65.0,...,Generally favorable,0,1,NaN,NaN,"Sakura Films,HBO Films,Rainmark Films,Home Box...",Stephen Frears,"Howard L. Bingham,Shawn Slovo,Mark Wallace","Christopher Plummer,Frank Langella,Ed Begley J...",True
1776,1000301445,Barbershop,NaN,1.0,TV-MA,Comedy,"Barbershop, based on the successful MGM featur...",30.0,NaN,64.0,...,Generally favorable,0,1,NaN,NaN,"Blind Decker Productions,Cube Vision,Internati...","John Ridley,Linda Mendoza,Lee Shallat Chemel,K...","John Ridley,Matt Wickline,Stacy A. Littlejohn,...","Omar Gooding,Gbenga Akinnagbe,Anna Brown,Wes C...",False


In [12]:
data.loc[data['title'] == 'Manhunt (2013)', 'releaseDate'] = '2013-07-17'

In [49]:
missing_value_summary(data)

,Number of Missing Values,Percent of Total Values
created_by,1605,48.43
tagline,1598,48.22
director,1137,34.31
rating,517,15.60
userscore_sentiment,450,13.58
duration,327,9.87
writer,242,7.30
production_companies,3,0.09
genres,2,0.06
description,1,0.03


In [51]:
data[data["writer"].isna()].head(3)

,id,title,releaseDate,seasonCount,rating,genres,description,duration,tagline,metascore,...,userscore,userscore_count,userscore_sentiment,created_by,production_companies,director,writer,top_cast,top_Cast,seasonCount_was_null
0,1000358361,Planet Earth: Blue Planet II,2017-10-29,1.0,TV-G,Documentary,"Airing simultaneously on AMC, BBC America, IFC...",50.0,Take a deep breath,97.0,...,82,178,Universal acclaim,NaN,"BBC Natural History Unit (NHU),BBC Studios,BBC...",James Honeyborne,NaN,"David Attenborough,Peter Drost,Roger Munns,Rog...","David Attenborough,Peter Drost,Roger Munns,Rog...",False
1,1000359012,America to Me,2018-08-26,1.0,TV-14,Documentary,The 10-part documentary series from Steve Jame...,60.0,NaN,96.0,...,59,75,Mixed or average,NaN,"Participant,Kartemquin Films,Starz,Starz,Nolo ...","Bing Liu,Kevin Shaw,Steve James,Rebecca Parrish",NaN,"Kendale McCoy,Charles Donalson Jr.,Ke'Shawn Ku...","Kendale McCoy,Charles Donalson Jr.,Ke'Shawn Ku...",False
8,1000357277,O.J.: Made in America,2016-05-20,1.0,TV-MA,"Documentary,Biography,Crime,History,Sport",The seven-and-a-half-hour documentary chronicl...,467.0,NaN,96.0,...,71,44,Generally favorable,NaN,"ESPN Films,Laylow Films,ESPN Films,ESPN Films,...",Ezra Edelman,NaN,"Willie Brown,Harry Edwards,John McKay,Harry Al...","Willie Brown,Harry Edwards,John McKay,Harry Al...",True


In [13]:
writer_nans = data['writer'].isna().sum()
writer_nans_and_doc = (data['writer'].isna() & data['genres'].str.contains('Documentary', na=False)).sum()


In [14]:
mask = data['writer'].isna() & data['genres'].str.contains('Documentary', na=False)
data[mask].head(5)

,id,title,releaseDate,seasonCount,rating,genres,description,duration,tagline,metascore,...,metascore_sentiment,userscore,userscore_count,userscore_sentiment,created_by,production_companies,director,writer,top_cast,seasonCount_was_null
0,1000358361,Planet Earth: Blue Planet II,2017-10-29,1.0,TV-G,Documentary,"Airing simultaneously on AMC, BBC America, IFC...",50.0,Take a deep breath,97.0,...,Universal acclaim,82,178,Universal acclaim,NaN,"BBC Natural History Unit (NHU),BBC Studios,BBC...",James Honeyborne,NaN,"David Attenborough,Peter Drost,Roger Munns,Rog...",False
1,1000359012,America to Me,2018-08-26,1.0,TV-14,Documentary,The 10-part documentary series from Steve Jame...,60.0,NaN,96.0,...,Universal acclaim,59,75,Mixed or average,NaN,"Participant,Kartemquin Films,Starz,Starz,Nolo ...","Bing Liu,Kevin Shaw,Steve James,Rebecca Parrish",NaN,"Kendale McCoy,Charles Donalson Jr.,Ke'Shawn Ku...",False
8,1000357277,O.J.: Made in America,2016-05-20,1.0,TV-MA,"Documentary,Biography,Crime,History,Sport",The seven-and-a-half-hour documentary chronicl...,467.0,NaN,96.0,...,Universal acclaim,71,44,Generally favorable,NaN,"ESPN Films,Laylow Films,ESPN Films,ESPN Films,...",Ezra Edelman,NaN,"Willie Brown,Harry Edwards,John McKay,Harry Al...",True
22,1000358605,The Zen Diaries of Garry Shandling,2018-03-26,1.0,TV-MA,"Documentary,Biography,Comedy,Drama",The two-part documentary by Judd Apatow about ...,259.0,NaN,90.0,...,Universal acclaim,71,10,Generally favorable,NaN,"Apatow Productions,Home Box Office (HBO),HBO Max",Judd Apatow,NaN,"Garry Shandling,Judd Apatow,Kevin Nealon,Mike ...",True
29,1000360288,The Last Dance,2020-04-19,1.0,TV-MA,"Documentary,Biography,History,Sport",The 10-part documentary series directed by Jas...,50.0,NaN,90.0,...,Universal acclaim,88,264,Universal acclaim,NaN,"ESPN Films,NBA Entertainment,Mandalay Sports M...",Jason Hehir,NaN,"Michael Jordan,Phil Jackson,Scottie Pippen,Dav...",False


In [15]:

print(f"Total writer NaNs: {writer_nans}")
print(f"Writer NaNs that are Documentaries: {writer_nans_and_doc}")
print(f"Percentage: {writer_nans_and_doc / writer_nans * 100:.1f}%")

Total writer NaNs: 242
Writer NaNs that are Documentaries: 137
Percentage: 56.6%


In [16]:
# 1. Both Documentary AND Reality-TV
mask_both = (data['writer'].isna() & 
             data['genres'].str.contains('Documentary', na=False) & 
             data['genres'].str.contains('Reality-TV', na=False))
data.loc[mask_both, 'writer'] = 'None - Documentary/Reality-TV'

# 2. Documentary only (remaining NaNs)
mask_doc = data['writer'].isna() & data['genres'].str.contains('Documentary', na=False)
data.loc[mask_doc, 'writer'] = 'None - Documentary'

# 3. Reality-TV only (remaining NaNs)
mask_reality = data['writer'].isna() & data['genres'].str.contains('Reality-TV', na=False)
data.loc[mask_reality, 'writer'] = 'None - Reality-TV'

In [134]:
#missing_value_summary(data)

In [17]:
# Talk-Show
mask_talk = data['writer'].isna() & data['genres'].str.contains('Talk-Show', na=False)
data.loc[mask_talk, 'writer'] = 'None - Talk-Show'

# Game-Show
mask_game = data['writer'].isna() & data['genres'].str.contains('Game-Show', na=False)
data.loc[mask_game, 'writer'] = 'None - Game-Show'

In [18]:
remaining_nans = data['writer'].isna().sum()
print(f"Remaining writer NaNs: {remaining_nans}")

Remaining writer NaNs: 12


In [100]:
#data[data['writer'].isna()]

In [19]:
data.loc[data['title'] == 'Generation Cryo', 'writer'] = 'None - Documentary'
data.loc[data['title'] == 'Modern Dads', 'writer'] = 'None - Reality'
data.loc[data['title'] == 'Ellen DeGeneres: For Your Approval', 'writer'] = 'None - Stand-up'
data.loc[data['title'] == 'Killing Fields', 'writer'] = 'None - Documentary'
data.loc[data['title'] == "Dane Cook's Tourgasm", 'writer'] = 'None - Stand-up'

In [107]:
#data[data['writer'].isna()]

In [20]:
data.loc[data['title'] == 'Star Wars: Visions', 'writer'] = 'George Lucas, Masahiko Ōtsuka, Yasumi Atarashi'
data.loc[data['title'] == "Marvel's M.O.D.O.K.", 'writer'] = 'Jordan Blum, Patton Oswalt'
data.loc[data['title'] == 'The Boys Presents: Diabolical', 'writer'] = 'Garth Ennis, Darick Robertson'
data.loc[data['title'] == 'The Agency (2024)', 'writer'] = 'Jez Butterworth, John-Henry Butterworth'
data.loc[data['title'] == 'Creepshow', 'writer'] = 'Greg Nicotero, John Esposito'
data.loc[data['title'] == 'Ten Percent', 'writer'] = 'John Morton, Ella Road'
data.loc[data['title'] == 'Black Rabbit', 'writer'] = 'Zach Baylin, Kate Susman'

In [21]:
# Check no more NaN writers
data['writer'].isna().sum()

np.int64(0)

In [133]:
#missing_value_summary(data)

In [113]:
data[data["director"].isna()].tail(5)

,id,title,releaseDate,seasonCount,rating,genres,description,duration,tagline,metascore,...,metascore_sentiment,userscore,userscore_count,userscore_sentiment,created_by,production_companies,director,writer,top_cast,seasonCount_was_null
3286,1000351308,Dr. Ken,2015-10-02,2.0,TV-PG,Comedy,Dr. Ken (Ken Jeong) is a medical practitioner ...,21.0,NaN,26.0,...,Generally unfavorable,31,82,Generally unfavorable,"John Fox,Ken Jeong,Jared Stern","Davis Entertainment,Old Charlie,ABC Signature,...",NaN,"John Fox,Ken Jeong,Jared Stern,Nicole Sun,Andy...","Ken Jeong,Suzy Nakamura,Tisha Campbell,Jonatha...",False
3291,1000359025,Insatiable,2018-08-10,2.0,TV-MA,"Comedy,Drama,Thriller",Disgraced-lawyer-turned-beauty-pageant-coach B...,45.0,A coming of rage story,25.0,...,Generally unfavorable,70,282,Generally favorable,Lauren Gussis,"CBS Television Studios,Ryan Seacrest Productio...",NaN,"Lauren Gussis,Jenina Kibuka","Dallas Roberts,Debby Ryan,Christopher Gorham,K...",False
3293,1000302033,Unan1mous,2006-03-22,1.0,NaN,"Game-Show,Reality-TV",Fox's reality show Unan1mous takes nine contes...,NaN,$1.5 million... 9 people... 1 bunker... and on...,24.0,...,Generally unfavorable,0,1,NaN,NaN,"3 Ball Productions,20th Century Fox Television...",NaN,None - Reality-TV,"Jeselle Hicks,Jamie Lane",False
3302,1000303369,Knight Rider,2008-02-17,1.0,TV-14,"Adventure,Drama,Fantasy,Mystery,Thriller",Mike Traceur is tracked down by his ex-girlfri...,60.0,An icon reborn.,21.0,...,Generally unfavorable,59,27,Mixed or average,NaN,"Dutch Oven,Universal Media Studios (UMS),Natio...",NaN,"Dave Andron,Glen A. Larson,Teresa Huang,Monica...","Justin Bruening,Deanna Russo,Paul Campbell,Smi...",False
3313,1000368285,All's Fair (2025),2025-11-04,2.0,TV-MA,Drama,A group of female divorce attorneys leave thei...,42.0,NaN,17.0,...,Overwhelming dislike,27,109,Generally unfavorable,"Jon Robin Baitz,Joseph Baken,Ryan Murphy","20th Television,Disney Television Studios,Old ...",NaN,"Jon Robin Baitz,Joseph Baken,Ryan Murphy","Kim Kardashian,Naomi Watts,Niecy Nash,Teyana T...",False


In [22]:
data['director'] = data['director'].fillna('Unknown')
data['created_by'] = data['created_by'].fillna('Unknown')

In [23]:
missing_value_summary(data)

,Number of Missing Values,Percent of Total Values
tagline,1598,48.22
rating,517,15.60
userscore_sentiment,450,13.58
duration,327,9.87
releaseDate,12,0.36
production_companies,3,0.09
genres,2,0.06
description,1,0.03
id,0,0.00
seasonCount,0,0.00


In [119]:
data[data["duration"].isna()]

,id,title,releaseDate,seasonCount,rating,genres,description,duration,tagline,metascore,...,metascore_sentiment,userscore,userscore_count,userscore_sentiment,created_by,production_companies,director,writer,top_cast,seasonCount_was_null
3,1000302375,The Staircase,2004-06,1.0,TV-MA,"Documentary,Crime,Drama",An 8-part documentary series about the celebra...,NaN,Did He Do It?,95.0,...,Universal acclaim,71,62,Generally favorable,Unknown,"Maha Productions,ABC News,Docurama,Netflix,Net...",Jean-Xavier de Lestrade,"Jean-Xavier de Lestrade,Nathalie Sobania","Michael Peterson,David Rudolf,Ron Guerette,Mar...",False
16,1000300328,My So-Called Life,1994-08-25,1.0,TV-14,"Comedy,Drama,Romance",My So-Called Life started in 1994 on ABC. It w...,NaN,You know how it feels.,92.0,...,Universal acclaim,84,48,Universal acclaim,Winnie Holzman,"The Bedford Falls Company,ABC Productions,ABC ...","Mark Piznarski,Scott Winant,Todd Holland","Winnie Holzman,Jill Gordon,Jason Katims,Ellen ...","Bess Armstrong,Wilson Cruz,Claire Danes,Devon ...",False
32,1000360156,Visible: Out on Television,2020-02-14,1.0,TV-MA,"Documentary,History","The documentary series narrated by Janet Mock,...",NaN,NaN,89.0,...,Universal acclaim,50,38,Mixed or average,Unknown,"Tripod Media,Apple TV+",Ryan White,None - Documentary,"Wilson Cruz,Wanda Sykes,Michaela Jaé (MJ) Rodr...",False
43,1000360340,A Parks and Recreation Special,2020-04-30,1.0,NaN,"Unknown,Comedy",Leslie Knope checks in with her friends while ...,NaN,We could all use a little Knope right now.,89.0,...,Universal acclaim,74,28,Generally favorable,"Greg Daniels,Michael Schur","3 Arts Entertainment,Deedle-Dee Productions,Fr...",Morgan Sackett,"Greg Daniels,Michael Schur,Megan Amram,Dave Ki...","Amy Poehler,Aziz Ansari,Nick Offerman,Aubrey P...",True
47,1000360599,City So Real,2020-10-29,1.0,TV-14,Documentary,The five-part documentary directed by Steve Ja...,NaN,NaN,93.0,...,Universal acclaim,64,32,Generally favorable,Unknown,Kartemquin Films,Steve James,None - Documentary,No Cast,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3299,1000303741,Momma's Boys,2008-11-12,1.0,NaN,Reality-TV,Ryan Seacrest produces this NBC reality projec...,NaN,NaN,19.0,...,Overwhelming dislike,0,2,NaN,Unknown,"Glassman Media,Ryan Seacrest Productions,Natio...",Tony Croll,Ryan Seacrest,"Lauren Potter,Michelle Celeste Assil,Cara Quic...",False
3300,1000302621,Sons Of Hollywood,2006-06-30,1.0,NaN,Reality-TV,This series follows three young men as they tr...,NaN,NaN,21.0,...,Generally unfavorable,0,3,NaN,Unknown,NaN,Alex Vais,"Bobby Heyward,Adam Moonves,Alex Vais,David Wei...","David Weintraub,Sean Stewart,Bobby Heyward,Jon...",False
3301,1000342450,I Wanna Marry Harry,2014-02,1.0,NaN,"Reality-TV,Romance",12 contestants from the US travel to England f...,NaN,NaN,23.0,...,Generally unfavorable,23,19,Generally unfavorable,Unknown,"Ryan Seacrest Productions,Zig Zag,Fox Network,...",Ashley S. Gorman,"Danny Fenton,Ryan Seacrest","Craig DeSilva,Paul Leonard,Matt Hicks,Kimberly...",False
3305,1000302391,Identity,2006-12-18,2.0,NaN,Game-Show,Join Penn Jillette of the legendary magician/c...,NaN,NaN,22.0,...,Generally unfavorable,0,2,NaN,Unknown,"GOLDer Productions,Reveille Productions,Valenc...",Michael Simon,None - Game-Show,"Penn Jillette,Stacy Kaiser,Joe Cipriano,Mark E...",False


In [120]:
mask_nan = data['duration'].isna()
duration_nans = mask_nan.sum()

doc = (mask_nan & data['genres'].str.contains('Documentary', na=False)).sum()
reality = (mask_nan & data['genres'].str.contains('Reality-TV', na=False)).sum()
talk = (mask_nan & data['genres'].str.contains('Talk-Show', na=False)).sum()
game = (mask_nan & data['genres'].str.contains('Game-Show', na=False)).sum()

print(f"Total duration NaNs: {duration_nans}")
print(f"\nBreakdown:")
print(f"Documentary: {doc}")
print(f"Reality-TV: {reality}")
print(f"Talk-Show: {talk}")
print(f"Game-Show: {game}")

any_category = (mask_nan & (
    data['genres'].str.contains('Documentary', na=False) |
    data['genres'].str.contains('Reality-TV', na=False) |
    data['genres'].str.contains('Talk-Show', na=False) |
    data['genres'].str.contains('Game-Show', na=False)
)).sum()

print(f"\nTotal in known categories: {any_category}")
print(f"Remaining unexplained: {duration_nans - any_category}")

Total duration NaNs: 327

Breakdown:
Documentary: 29
Reality-TV: 52
Talk-Show: 4
Game-Show: 9

Total in known categories: 85
Remaining unexplained: 242


In [151]:
print(f"Duration NaNs: {data['duration'].isna().sum()}")

Duration NaNs: 327


In [152]:
missing_value_summary(data)

,Number of Missing Values,Percent of Total Values
tagline,1598,48.22
rating,517,15.60
userscore_sentiment,450,13.58
duration,327,9.87
production_companies,3,0.09
genres,2,0.06
description,1,0.03
title,0,0.00
id,0,0.00
seasonCount,0,0.00


In [157]:
#data[data["rating"].isna()].head(10)

In [24]:
data['rating'] = data['rating'].fillna('Unknown')

In [25]:
missing_value_summary(data)

,Number of Missing Values,Percent of Total Values
tagline,1598,48.22
userscore_sentiment,450,13.58
duration,327,9.87
releaseDate,12,0.36
production_companies,3,0.09
genres,2,0.06
description,1,0.03
title,0,0.00
id,0,0.00
rating,0,0.00


In [26]:
data[data["userscore_sentiment"].isna()].head(10)

,id,title,releaseDate,seasonCount,rating,genres,description,duration,tagline,metascore,...,metascore_sentiment,userscore,userscore_count,userscore_sentiment,created_by,production_companies,director,writer,top_cast,seasonCount_was_null
106,1000342394,The Trip to Bountiful,2014-03-08,1.0,TV-PG,Drama,Carrie Watts (Cicely Tyson) is a widow frustra...,102.0,NaN,86.0,...,Universal acclaim,0,1,NaN,Unknown,"Ostar Productions,Lifetime Television,Moviemax...",Michael Wilson,Horton Foote,"Cicely Tyson,Vanessa Williams,Blair Underwood,...",True
131,1000342327,Six by Sondheim,2013-12-09,1.0,Unknown,"Documentary,Biography,History,Music","Newly filmed performances from Darren Criss, J...",86.0,NaN,85.0,...,Universal acclaim,0,1,NaN,Unknown,"Sabella Entertainment,Home Box Office (HBO),HB...","James Lapine,Autumn de Wilde,Todd Haynes",None - Documentary,"Barbara Barrie,Jack Bashkow,Judy Bennett,Charl...",True
151,1000359541,What's My Name: Muhammad Ali,2019-05-15,1.0,TV-14,"Documentary,Biography,Sport",The two-part documentary about the iconic boxe...,165.0,NaN,85.0,...,Universal acclaim,0,2,NaN,Unknown,"Sutter Road Picture Company,Hill District Medi...",Antoine Fuqua,Steven Leckart,"Bill Cosby,Jim Brown,Kareem Abdul-Jabbar,Diana...",True
153,1000342001,Wonderland,2000-03-30,1.0,Unknown,Drama,"""With millions packed into New York City, ther...",50.0,NaN,85.0,...,Universal acclaim,0,0,NaN,Unknown,"Touchstone Television,Imagine Television,Hosta...","Peter Berg,Adam Bernstein,John David Coles,Oz ...","Peter Berg,Mark Leyner,Guy Zimmerman,Bonnie Ma...","Ted Levine,Michelle Forbes,Martin Donovan,Shar...",False
166,1000341929,EZ Streets,1996-10-27,1.0,TV-14,"Crime,Drama,Romance","EZ Streets is an epic drama about a dark, grit...",45.0,NaN,84.0,...,Universal acclaim,0,2,NaN,Paul Haggis,"Paul Haggis Productions,Universal Television,C...","Paul Haggis,Ken Olin","Paul Haggis,Patrick Harbinson,Bobby Moresco,Da...","Ken Olin,Joe Pantoliano,Jason Gedrick,Mike Sta...",False
167,1000357192,The Dresser (2016),2015-10-31,1.0,TV-14,Drama,"During World War II, a small English regional ...",105.0,NaN,84.0,...,Universal acclaim,0,2,NaN,Unknown,"Playground Entertainment,Sonia Friedman Produc...",Richard Eyre,"Richard Eyre,Ronald Harwood","Anthony Hopkins,Ian McKellen,Emily Watson,Sara...",True
183,1000359690,No Direction Home: Bob Dylan,2005-09-26,1.0,Unknown,"Documentary,Biography,History,Music",Martin Scorsese's two-part documentary about l...,208.0,NaN,84.0,...,Universal acclaim,0,3,NaN,Unknown,"Spitfire Pictures,Grey Water Park Productions,...",Martin Scorsese,None - Documentary,"Bob Dylan,B.J. Rolfzen,Dick Kangas,Liam Clancy...",True
209,1000342313,Whoopi Goldberg Presents Moms Mabley,NaN,1.0,Unknown,"Documentary,Comedy","With newly discovered archival footage, photog...",72.0,NaN,83.0,...,Universal acclaim,0,1,NaN,Unknown,Whoop/One Ho Productions/Lil' Whoop Production...,Whoopi Goldberg,None - Documentary,"Debbie Allen,Kaye Ballard,Harry Belafonte,Dick...",True
245,1000338235,Wartorn: 1861-2010,2010-11-11,1.0,Unknown,"Documentary,War",This documentary film explores combat stress a...,68.0,NaN,82.0,...,Universal acclaim,0,1,NaN,Unknown,"Attaboy Films,HBO Documentary Films,Home Box O...","Jon Alpert,Ellen Goosenberg Kent,Matthew O'Neill",None - Documentary,No Cast,True
246,1000357614,Marathon: The Patriot's Day Bombing,2016-11-18,1.0,TV-MA,Documentary,The Ricki Stern and Annie Sundberg documentary...,108.0,NaN,82.0,...,Universal acclaim,0,0,NaN,Unknown,"HBO Documentary Films,Break Thru Films,Home Bo...","Ricki Stern,Anne Sundberg","Ricki Stern,Anne Sundberg","Jeff Bauman,Kelly Castine,Celeste Corcoran,Syd...",True


In [27]:
# Verify: all userscore_sentiment NaNs have userscore_count < 4
mask_nan = data['userscore_sentiment'].isna()
low_count = data.loc[mask_nan, 'userscore_count'] < 4

print(f"Total userscore_sentiment NaNs: {mask_nan.sum()}")
print(f"Of these, userscore_count < 4: {low_count.sum()}")
print(f"Exceptions (NaN but count >= 4): {mask_nan.sum() - low_count.sum()}")

Total userscore_sentiment NaNs: 450
Of these, userscore_count < 4: 450
Exceptions (NaN but count >= 4): 0


In [28]:
# Pick your preferred
data['userscore_sentiment'] = data['userscore_sentiment'].fillna('Insufficient scores')

In [31]:
data[data["production_companies"].isna()]

,id,title,releaseDate,seasonCount,rating,genres,description,duration,tagline,metascore,...,metascore_sentiment,userscore,userscore_count,userscore_sentiment,created_by,production_companies,director,writer,top_cast,seasonCount_was_null
1509,1000339026,Citizen U.S.A.: A 50 State Road Trip,2011,1.0,TV-PG,Documentary,Director Alexandra Pelosi travels across Ameri...,51.0,NaN,66.0,...,Generally favorable,0,0,Insufficient scores,Unknown,NaN,Alexandra Pelosi,None - Documentary,"Alexandra Pelosi,Ghina Abul-Khoudoud,Hassan Ab...",True
3030,1000337779,Breakthrough with Tony Robbins,2010-07-27,2.0,Unknown,Reality-TV,Self-help guru Tony Robbins is coming to NBC. ...,NaN,NaN,43.0,...,Mixed or average,0,1,Insufficient scores,Unknown,NaN,Brian Smith,None - Reality-TV,Anthony Robbins,False
3300,1000302621,Sons Of Hollywood,2006-06-30,1.0,Unknown,Reality-TV,This series follows three young men as they tr...,NaN,NaN,21.0,...,Generally unfavorable,0,3,Insufficient scores,Unknown,NaN,Alex Vais,"Bobby Heyward,Adam Moonves,Alex Vais,David Wei...","David Weintraub,Sean Stewart,Bobby Heyward,Jon...",False


In [29]:
missing_value_summary(data)

,Number of Missing Values,Percent of Total Values
tagline,1598,48.22
duration,327,9.87
releaseDate,12,0.36
production_companies,3,0.09
genres,2,0.06
description,1,0.03
seasonCount,0,0.00
title,0,0.00
id,0,0.00
rating,0,0.00
